In [1]:
from bs4 import BeautifulSoup
from gensim.models import CoherenceModel
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import gensim.corpora as corpora
import gensim
import html
import itertools
import networkx as nx
import nltk
import numpy as np
import pandas as pd
import re

c:\Users\pih\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
def add_nodes(graph, users):
    for _, user in users.iterrows():
        attributes = user.to_dict()
        user_id = attributes.pop('Id')
        graph.add_node(user_id, **attributes)

def add_answer_edges(graph, posts):
    answers = posts[(posts['PostTypeId'] == 2) & posts['ParentId'].notna() & posts['OwnerUserId'].notna()]
    questions = posts[posts['PostTypeId'] == 1].set_index('Id')['OwnerUserId']

    for answerer, question in zip(answers['OwnerUserId'], answers['ParentId']):
        asker = questions.get(question)
        if asker and answerer != asker:
            graph.add_edge(answerer, asker)

def add_comment_edges(graph, comments, posts):
    comments = comments.dropna(subset=['PostId', 'UserId'])
    post_owners = posts.set_index('Id')['OwnerUserId']

    for commenter, post in zip(comments['UserId'], comments['PostId']):
        owner = post_owners.get(post)
        if owner and commenter != owner:
            graph.add_edge(commenter, owner)

def filter_active_users(users, posts, comments, min_activity):
    post_counts = posts['OwnerUserId'].value_counts()
    comment_counts = comments['UserId'].value_counts()
    total_activity = post_counts.add(comment_counts, fill_value=0)

    active_users = total_activity[total_activity >= min_activity].index
    return users[users['Id'].isin(active_users)]

def filter_active_posts(posts, active_users):
    return posts[posts['OwnerUserId'].isin(active_users['Id'])]

def filter_active_comments(comments, active_users, active_posts):
    return comments[comments['UserId'].isin(active_users['Id']) & comments['PostId'].isin(active_posts['Id'])]

